# Desafío 03

## Integrantes

- Acevedo Zain, Gaspar (acevedo.zain.gaspar@gmail.com)

## Consignas

- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


***Sugerencias***
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


## Imports

In [31]:
import random
import io
import pickle

import os
import platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from scipy.special import softmax
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import pad_sequences

In [2]:
import urllib.request
import bs4 as bs

## Selección del corpus

El objetivo de esta práctica es evaluar modelos de lenguajes con *tokenización por caracteres*, por lo cual, un texto lo suficientemente grande puede servir como `Corpus`.

Se elige entonces el libro `La Odisea` de Homero ([source](https://www.textos.info/homero/odisea/ebook)) como `Corpus`.

In [3]:
odisea_url = "https://www.textos.info/homero/odisea/ebook"

In [4]:
raw_html = urllib.request.urlopen(odisea_url)
raw_html = raw_html.read()

Se procesa el *html* original mediante la utilidad `bs.BeautifulSoup` a fin de tener el texto en ***un solo string***.

In [5]:
article_html = bs.BeautifulSoup(raw_html, 'lxml')
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text + ' '

article_text = article_text.lower()

Se muestran los primeros $500$ caracteres del texto (corpus).

In [6]:
article_text[:500]

' háblame, musa, de aquel varón de multiforme ingenio que, después de \ndestruir la sacra ciudad de troya, anduvo peregrinando larguísimo \ntiempo, vio las poblaciones y conoció las costumbres de muchos hombres y\n padeció en su ánimo gran número de trabajos en su navegación por el \nponto, en cuanto procuraba salvar su vida y la vuelta de sus compañeros a\n la patria. mas ni aun así pudo librarlos, como deseaba, y todos \nperecieron por sus propias locuras. ¡insensatos! comiéronse las vacas de\n helios'

En el texto se observan algunas *secuencias de escape de caracteres* (salto de línea o `\n`), por lo cual se las reemplaza a continuación con un caracter vacío.

In [7]:
article_text = str.replace(article_text, "\n", "")

In [8]:
article_text[:500]

' háblame, musa, de aquel varón de multiforme ingenio que, después de destruir la sacra ciudad de troya, anduvo peregrinando larguísimo tiempo, vio las poblaciones y conoció las costumbres de muchos hombres y padeció en su ánimo gran número de trabajos en su navegación por el ponto, en cuanto procuraba salvar su vida y la vuelta de sus compañeros a la patria. mas ni aun así pudo librarlos, como deseaba, y todos perecieron por sus propias locuras. ¡insensatos! comiéronse las vacas de helios, hijo '

## Definición del vocabulario + Tokenización

En esta sección definiremos nuestro `vocabulario` a partir del corpus original.

Luego lo tokenizaremos, a fin de que pueda ser procesado por una Red Neuronal en pasos posteriores.

Comenzamos definiendo nuestro vocabulario, que es el conjunto de distintos caracteres que aparecen en nuestro corpus.

Para el caso particular del texto seleccionado el tamaño es de $57$.

In [9]:
char_vocab = set(article_text)
print(f"Tamaño del vocabulario: {len(char_vocab)}")

Tamaño del vocabulario: 57


Definimos dos diccionarios que se utilizarán a lo largo de este trabajo:
- `char2idx`: A cada caracter de nuestro vocabulario se le asigna un `índice`.
- `idx2char`: Es el *inverso* de `char2idx`, es decir, dado un índice, me devuelve el caracter correspondiente.

In [13]:
char2idx = {k: v for v,k in enumerate(char_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

Ahora `tokenizamos` el corpus. Para ello hacemos uso del diccionario `char2idx`, reemplazando cada caracter por su índice correspondiente.

In [17]:
tokenized_text = [char2idx[ch] for ch in article_text]

Para validar, mostramos primero los $10$ primero caracteres tokenizados del corpus/texto, correspondientes a **" háblame, "** (nótesen los espacios en blanco al inicio y al final):

In [21]:
tokenized_text[:10]

[15, 49, 45, 50, 18, 33, 47, 34, 44, 15]

## Definición del dataset

En esta sección definimos los datos de entrenamiento y validación.

Primero se definien las siguientes constantes:
- `MAX_CONTEXT_SIZE`: corresponde al tamaño máximo del contexto que se analizará. Se define inicialmente en $100$.
- `VALIDATION_SIZE`: tamaño del set de validación. En este caso, se opta por utilizar el $30\%$.

In [23]:
MAX_CONTEXT_SIZE = 100
VALIDATION_SIZE = 0.3

Se define también la cantidad de secuencias de tamaño `MAX_CONTENT_SIZE` que tendrá el set de validación mediante la variable `SEQ_VALIDATION`.

Dado a que el tamaño del texto tokenizado es $673064$, el valor de `SEQ_VALIDATION` queda en $2020$, es decir, habrá en el set de validación un total de $2020$ secuencias de tamaño máximo $100$.

In [24]:
SEQ_VALIDATION = int(np.ceil(len(tokenized_text)*VALIDATION_SIZE/MAX_CONTEXT_SIZE))

Se realiza la separación del corpus original en `train` (train_text) y `validation` (val_text).

In [28]:
train_text = tokenized_text[:-SEQ_VALIDATION*MAX_CONTEXT_SIZE]
val_text = tokenized_text[-SEQ_VALIDATION*MAX_CONTEXT_SIZE:]

In [29]:
tokenized_sentences_val = [val_text[init*MAX_CONTEXT_SIZE:init*(MAX_CONTEXT_SIZE+1)] for init in range(SEQ_VALIDATION)]
tokenized_sentences_train = [train_text[init:init+MAX_CONTEXT_SIZE] for init in range(len(train_text)-MAX_CONTEXT_SIZE+1)]

In [30]:
X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

Validamos los tamaños de `X` e `y`.

In [43]:
X.shape

(470964, 100)

In [49]:
y.shape

(470964, 100)